<a href="https://colab.research.google.com/github/waltatgit/CTCModel/blob/master/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
from google.colab import drive
ROOT = '/content/drive'
drive.mount(ROOT)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
%cd '/content/drive/My Drive/Colab Notebooks/CTCModel'

/content/drive/My Drive/Colab Notebooks/CTCModel


In [15]:
!tar xvf ./seqDigits.pkl.tar.gz

tar: /tmp/seqDigits.pk: Not found in archive
tar: Exiting with failure status due to previous errors


In [7]:
import sys
sys.path.append('/content/drive/My Drive/Colab Notebooks/CTCModel')

In [28]:
from keras.layers import TimeDistributed, Activation, Dense, Input, Bidirectional, LSTM, Masking, GaussianNoise
from CTCModel import CTCModel
import pickle
from keras.preprocessing import sequence
import numpy as np

In [9]:
def create_network(nb_features, nb_labels, padding_value):

    # Define the network architecture
    input_data = Input(name='input', shape=(None, nb_features)) # nb_features = image height

    masking = Masking(mask_value=padding_value)(input_data)
    noise = GaussianNoise(0.01)(masking)
    blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(noise)
    blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)
    blstm = Bidirectional(LSTM(128, return_sequences=True, dropout=0.1))(blstm)

    dense = TimeDistributed(Dense(nb_labels + 1, name="dense"))(blstm)
    outrnn = Activation('softmax', name='softmax')(dense)

    network = CTCModel([input_data], [outrnn])
    network.compile(Adam(lr=0.0001))

    return network

In [10]:
    (x_train, y_train), (x_test, y_test) = pickle.load(open('./seqDigits.pkl', 'rb'))


In [11]:
    nb_labels = 10 # number of labels (10, this is digits)
    batch_size = 32 # size of the batch that are considered
    padding_value = 255 # value for padding input observations
    nb_epochs = 10 # number of training epochs
    nb_train = len(x_train)
    nb_test = len(x_test)
    nb_features = len(x_train[0][0])

In [12]:
    # create list of input lengths
    x_train_len = np.asarray([len(x_train[i]) for i in range(nb_train)])
    x_test_len = np.asarray([len(x_test[i]) for i in range(nb_test)])
    y_train_len = np.asarray([len(y_train[i]) for i in range(nb_train)])
    y_test_len = np.asarray([len(y_test[i]) for i in range(nb_test)])

    # pad inputs
    x_train_pad = sequence.pad_sequences(x_train, value=float(padding_value), dtype='float32',
                                         padding="post", truncating='post')
    x_test_pad = sequence.pad_sequences(x_test, value=float(padding_value), dtype='float32',
                                        padding="post", truncating='post')
    y_train_pad = sequence.pad_sequences(y_train, value=float(nb_labels),
                                         dtype='float32', padding="post")
    y_test_pad = sequence.pad_sequences(y_test, value=float(nb_labels),
                                        dtype='float32', padding="post")

In [30]:
    # define a recurrent network using CTCModel
    network = create_network(nb_features, nb_labels, padding_value)

In [14]:
 # CTC training
network.fit(x=[x_train_pad, y_train_pad, x_train_len, y_train_len], y=np.zeros(nb_train), \
                batch_size=batch_size, epochs=nb_epochs)



Epoch 1/10
10000/10000 [==============================] - 365s 36ms/step - loss: 16.1104
Epoch 2/10
10000/10000 [==============================] - 360s 36ms/step - loss: 8.1164
Epoch 3/10
10000/10000 [==============================] - 365s 37ms/step - loss: 7.2863
Epoch 4/10
10000/10000 [==============================] - 361s 36ms/step - loss: 6.1459
Epoch 5/10
10000/10000 [==============================] - 357s 36ms/step - loss: 4.9325
Epoch 6/10
10000/10000 [==============================] - 358s 36ms/step - loss: 3.4907
Epoch 7/10
10000/10000 [==============================] - 353s 35ms/step - loss: 2.0569
Epoch 8/10
10000/10000 [==============================] - 354s 35ms/step - loss: 1.4629
Epoch 9/10
10000/10000 [==============================] - 355s 35ms/step - loss: 1.1664
Epoch 10/10
10000/10000 [==============================] - 358s 36ms/step - loss: 0.9749


In [18]:
# Evaluation: loss, label error rate and sequence error rate are requested
eval = network.evaluate(x=[x_test_pad, y_test_pad, x_test_len, y_test_len],\
                            batch_size=batch_size, metrics=['loss', 'ler', 'ser'])

2000/2000 [==============================] - 23s 11ms/step


In [19]:
# predict label sequences
pred = network.predict([x_test_pad, x_test_len], batch_size=batch_size, max_value=padding_value)
for i in range(10):  # print the 10 first predictions
  print("Prediction :", [j for j in pred[i] if j!=-1], " -- Label : ", y_test[i]) # 

Prediction : [3.0, 3.0, 1.0, 4.0]  -- Label :  [3. 3. 1. 4.]
Prediction : [1.0, 1.0]  -- Label :  [1. 1.]
Prediction : [0.0, 7.0, 2.0]  -- Label :  [0. 7. 2.]
Prediction : [6.0, 4.0]  -- Label :  [6. 4.]
Prediction : [5.0, 8.0]  -- Label :  [5. 8.]
Prediction : [0.0, 6.0, 9.0]  -- Label :  [0. 6. 9.]
Prediction : [6.0, 8.0, 6.0, 6.0]  -- Label :  [6. 8. 6. 6.]
Prediction : [4.0, 1.0]  -- Label :  [4. 1.]
Prediction : [8.0, 4.0]  -- Label :  [8. 4.]
Prediction : [2.0, 3.0, 3.0, 1.0]  -- Label :  [2. 3. 3. 1.]
